The goal of this notebook is to test if a reconstruction learned on VIKING20X data can be transferred to the rapid prediction.

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd

/gxfs_home/geomar/smomw507/.conda/envs/amoc_recons/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [32]:
std_input = True
temporal_resmaple = '1D'

sim_dataset = np.load(f'../rapid-geostrophic-reconstruction/data/learning/ds_pos_sim_miss{"_std" if std_input else ""}_t_umo_sim_miss_{temporal_resmaple}.npz')
obs_dataset = np.load(f'../rapid-geostrophic-reconstruction/data/learning/ds_pos_obs{"_std" if std_input else ""}_t_umo_obs_{temporal_resmaple}.npz')

list(sim_dataset.keys())

['numpy_values', 'numpy_time', 'numpy_z', 'numpy_umo']

In [33]:
train_X = sim_dataset['numpy_values']
train_y = sim_dataset['numpy_umo']

test_X = obs_dataset['numpy_values']
test_y = obs_dataset['numpy_umo']

In [34]:
# lasso = RidgeCV(alphas=np.logspace(-8, 0, 9)).fit(train_X, train_y)
lasso = Ridge(alpha=1e-2).fit(train_X, train_y)

In [35]:
train_prediction = lasso.predict(train_X)
test_prediction = lasso.predict(test_X)


In [36]:
result_dataframe_train = pd.DataFrame({
    'time': sim_dataset['numpy_time'],
    'true': train_y,
    'prediction': train_prediction,
})

result_dataframe_test = pd.DataFrame({
    'time': obs_dataset['numpy_time'],
    'true': test_y,
    'prediction': test_prediction,
    'difference': test_y - test_prediction,
})



In [37]:
(
    result_dataframe_train.hvplot.line(x='time', y=['true', 'prediction'])
    + result_dataframe_test.hvplot.line(x='time', y=['true', 'prediction'])
    + result_dataframe_test.hvplot.line(x='time', y='difference')
).cols(1)

/gxfs_home/geomar/smomw507/.conda/envs/amoc_recons/lib/python3.9/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/gxfs_home/geomar/smomw507/.conda/envs/amoc_recons/lib/python3.9/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/gxfs_home/geomar/smomw507/.conda/envs/amoc_recons/lib/python3.9/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will alwa

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [time]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [time]   (value)
   .Curve.I      :Curve   [time]   (difference)

In [38]:
# Scores 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error

print('---- Train ----')
print(f'MSE: {mean_squared_error(train_y, train_prediction):6.2f}')
print(f'R2: {r2_score(train_y, train_prediction)*100.0:7.2f}%')
print(f'MAPE: {mean_absolute_percentage_error(train_y, train_prediction)* 100.0:5.2f}%')
print(f'MAE: {mean_absolute_error(train_y, train_prediction):6.2f}')

print('---- Test ----')
print(f'MSE: {mean_squared_error(test_y, test_prediction):6.2f}')
print(f'R2: {r2_score(test_y, test_prediction)*100.0:7.2f}%')
print(f'MAPE: {mean_absolute_percentage_error(test_y, test_prediction)*100.0:5.2f}%')
print(f'MAE: {mean_absolute_error(test_y, test_prediction):6.2f}')


---- Train ----
MSE:   0.06
R2:   99.57%
MAPE:  1.15%
MAE:   0.20
---- Test ----
MSE:   2.00
R2:   92.88%
MAPE:  9.98%
MAE:   1.23
